In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
import sys
sys.path.append('../')

from myutils.metrics import mapk

OUTPUT_NAMEと、item_vecs = make_prone_vec(train_test_log_df, dimensions)を変えるだけ

In [3]:
OUTPUT_NAME = 'prone_test_only'

In [4]:
train_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_log.csv')
train_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_label.csv')
test_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_log.csv')
yado_df = pd.read_csv('../datasets/atmaCup16_Dataset/yado.csv')

train_session_only_df = train_df[['session_id']]
test_session_only_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_session.csv')

train_test_log_df = pd.concat([train_log_df, test_log_df], axis=0).reset_index(drop=True)

In [5]:
yado_df['yad_no'].min()

1

In [6]:
from csrgraph import csrgraph
import nodevectors as nv
from scipy.sparse import csr_matrix
from nodevectors.prone import ProNE
from nodevectors import prone

def create_adjacency_matrix(edges):
    """
    無向グラフ
    """
    n = np.max(edges) + 1  # ノードの数を取得します（ノードが0から始まると仮定します）

    # edges_flipped = np.flip(edges, axis=1)  # エッジの始点と終点を入れ替えます
    # edges_doubled = np.vstack((edges, edges_flipped))  # エッジリストを2倍にします
    data = np.ones(len(edges))  # エッジの数だけ1を用意します
    adjacency_matrix = csr_matrix((data, (edges[:, 0], edges[:, 1])), shape=(n, n))  # CSR形式のスパース行列を作成します

    return adjacency_matrix


def calc_prone_embs(edge, n_components):
    # 隣接行列
    adj_mat = create_adjacency_matrix(edge)
    # グラフ
    graph = csrgraph(adj_mat)

    # 埋め込みモデルの学習
    model_params = dict(n_components=n_components, step=10, mu=0.2, theta=0.5, exponent=0.75, verbose=True)
    node_emb_model = prone.ProNE(**model_params)
    node_embs = node_emb_model.fit_transform(graph)
    # 正規化する
    node_embs = node_embs / (np.linalg.norm(node_embs, axis=1, keepdims=True) + 1e-8)
    return node_embs


def make_prone_vec(train_test_log_df: pd.DataFrame, n_components:int):
    # 視聴グラフを作成
    # エッジ。anime_noがuser_noの後に続くように調整
    
    # view_edge = df[[f"{name1}_no", f"{name2}_no"]].values
    # num_user = np.max(view_edge[:, 0]) + 1
    # view_edge[:, 1] = view_edge[:, 1] + num_user
    sentence_list = train_test_log_df.groupby('session_id')['yad_no'].apply(list).tolist()

    edges = set()
    for items in sentence_list:
        if len(items) == 1:
            continue
        for i in range(len(items)-1):
            edges.add((items[i]-1, items[i+1]-1))
            edges.add((items[i+1]-1, items[i]-1))
    edges = list(edges)
    edges = np.array(edges)
    print(edges.shape) # (num_edge, 2)

    node_embs = calc_prone_embs(edges, n_components)

    return node_embs

In [7]:
dimensions = 1024

In [8]:
# item_vecs = make_prone_vec(train_test_log_df, dimensions)
item_vecs = make_prone_vec(test_log_df, dimensions)

(56676, 2)


In [9]:
item_vecs.shape

(13806, 1024)

In [10]:
# item_ids = train_test_log_df['yad_no'].unique().tolist()
item_ids = yado_df['yad_no'].to_list()
item_factors = {item: item_vecs[item-1, :] for item in item_ids}
item_factors_df = pd.DataFrame(item_factors).T.reset_index().rename(columns={"index": "yad_no"})

item_cols = [f"item_factor_{i}" for i in range(dimensions)]
item_factors_df.columns = ["yad_no"] + item_cols

In [11]:
user_item_list_dict = {session_id: yad_nos.tolist() for session_id, yad_nos in train_test_log_df.groupby('session_id')['yad_no']}
user_factors = {user_id: np.mean([item_factors[item_id] for item_id in user_item_list], axis=0) for user_id, user_item_list in user_item_list_dict.items()}
user_factors_df = pd.DataFrame(user_factors).T.reset_index().rename(columns={"index": "session_id"})
user_cols = [f"user_factor_{i}" for i in range(dimensions)]
user_factors_df.columns = ["session_id"] + user_cols

user_factors_last_item = {user_id: item_factors[user_item_list[-1]] for user_id, user_item_list in user_item_list_dict.items()}
user_factors_last_item_df = pd.DataFrame(user_factors_last_item).T.reset_index().rename(columns={"index": "session_id"})
user_cols = [f"user_factor_{i}" for i in range(dimensions)]
user_factors_last_item_df.columns = ["session_id"] + user_cols

In [12]:
item_factors_df

,yad_no,item_factor_0,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,...,item_factor_1014,item_factor_1015,item_factor_1016,item_factor_1017,item_factor_1018,item_factor_1019,item_factor_1020,item_factor_1021,item_factor_1022,item_factor_1023
0,1,-7.338344e-05,0.000110,-0.000041,3.208105e-05,-2.645909e-06,0.000073,0.106289,0.000171,-0.000012,...,-0.051621,-0.016698,-0.001650,0.020203,-0.004682,-0.016487,-0.023659,0.014572,-0.000567,0.001257
1,2,-2.454955e-05,-0.000010,0.000001,9.978755e-07,2.017953e-07,0.000033,-0.000003,0.000023,-0.000021,...,0.001030,-0.000140,0.001950,0.000219,0.003382,0.008148,0.004206,-0.006679,-0.003000,-0.014547
2,3,-9.746580e-02,-0.001218,0.000059,6.703441e-04,1.853147e-05,-0.000645,-0.000971,-0.000534,-0.000023,...,-0.008966,-0.001162,-0.007610,-0.000940,0.012517,-0.011451,0.011339,-0.003595,0.006617,0.008104
3,4,-1.385527e-02,-0.000451,-0.000005,5.134649e-05,-1.197618e-05,-0.000126,-0.000402,0.000055,0.000058,...,0.002379,0.004047,-0.002729,-0.000034,0.000502,-0.005691,-0.000911,-0.002261,-0.000919,0.009346
4,5,-2.024649e-04,-0.000736,0.000057,1.493503e-03,-3.055729e-03,0.000310,0.003308,0.000788,0.000841,...,-0.044515,0.021466,-0.015693,0.065556,-0.028983,0.017788,0.042445,0.007244,0.019732,0.048272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13801,13802,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13802,13803,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13803,13804,-3.443516e-05,0.000088,0.000003,4.077758e-06,-4.925227e-05,0.000020,0.005261,0.000046,0.000059,...,0.012638,-0.050222,0.001002,0.041713,0.043683,0.045328,0.072325,-0.145837,-0.036217,0.143727
13804,13805,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
user_factors_df

,session_id,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,...,user_factor_1014,user_factor_1015,user_factor_1016,user_factor_1017,user_factor_1018,user_factor_1019,user_factor_1020,user_factor_1021,user_factor_1022,user_factor_1023
0,000007603d533d30453cc45d0f3d119f,-0.000049,-8.762568e-05,-0.000200,-0.000019,0.000062,-0.000013,-0.000072,-0.000083,0.000114,...,0.001022,0.000518,-0.004136,-0.002956,-0.002306,-0.000476,0.000415,0.000152,0.001061,-0.001134
1,00001149e9c73985425197104712478c,-0.000190,7.152080e-04,-0.000015,0.000010,-0.000057,0.000004,0.000556,-0.000197,0.000116,...,-0.002879,-0.001697,-0.002982,0.001607,-0.014424,-0.001202,-0.001464,0.007898,0.006756,0.010042
2,0000ca043ed437a1472c9d1d154eb49b,0.000076,9.022307e-05,0.000039,-0.000033,-0.000107,0.000087,-0.000050,0.000065,-0.000095,...,-0.011601,0.010087,0.010686,0.010533,-0.003031,-0.113358,0.069297,-0.037588,-0.028360,-0.035655
3,0000d4835cf113316fe447e2f80ba1c8,-0.000288,3.658835e-04,0.000181,0.000941,0.000600,0.000415,-0.000254,0.000286,-0.000050,...,0.064076,0.050815,0.059042,0.008956,0.037928,-0.018577,0.036259,0.012071,-0.080678,-0.059647
4,0000e02747d749a52b7736dfa751e258,-0.062572,5.328496e-04,0.000034,0.000585,-0.000032,0.000057,-0.000764,-0.000016,0.000094,...,0.008412,0.004585,0.001297,0.009607,-0.017946,-0.006185,0.002853,-0.004413,-0.006943,0.009387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463393,ffff9a7dcc892875c7a8b821fa436228,-0.011073,2.664768e-01,-0.000035,-0.000089,-0.000066,0.000130,0.001005,-0.000099,-0.000011,...,0.006583,0.007268,0.003360,0.002350,-0.003145,-0.000605,0.015077,-0.001597,0.004091,0.013504
463394,ffffb1d30300fe17f661941fd085b04b,-0.007966,5.885928e-03,0.000053,0.000090,0.000022,-0.000007,0.008899,-0.000269,0.000405,...,0.011836,-0.001815,0.024678,0.028244,-0.003282,-0.015438,-0.017916,-0.006375,-0.003611,0.006625
463395,ffffcd5bc19d62cad5a3815c87818d83,-0.000028,9.351237e-08,-0.000017,-0.000014,0.000057,0.000146,-0.000004,-0.000007,-0.000008,...,-0.001177,-0.002766,0.002234,-0.005575,0.004034,-0.007609,0.007865,-0.004872,-0.004836,-0.006538
463396,ffffe984aafd6127ce8e43e3ca40c79d,-0.008678,8.315513e-03,0.000068,0.000094,0.000074,-0.000058,0.000402,-0.000082,0.000090,...,0.012913,-0.011774,-0.012405,0.008745,0.009246,-0.007780,-0.038557,-0.011695,-0.035317,-0.005866


In [14]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [15]:
from sklearn.neighbors import NearestNeighbors


user_vecs = user_factors_df[[f'user_factor_{i}' for i in range(dimensions)]].values
# user_vecs = user_factors_last_item_df[[f'user_factor_{i}' for i in range(dimensions)]].values
item_vecs = item_factors_df[[f'item_factor_{i}' for i in range(dimensions)]].values

nn = NearestNeighbors(n_neighbors=20, metric='cosine')
nn.fit(item_vecs)
dists, indices = nn.kneighbors(user_vecs)

In [16]:
user_last_item_dict = {user_id: user_item_list[-1] for user_id, user_item_list in user_item_list_dict.items()}

In [17]:
results = []

for i, user_id in enumerate(user_item_list_dict.keys()):
    for j in range(20):
        yad_no = indices[i][j]+1
        if user_last_item_dict[user_id] == yad_no:
            continue

        results.append([user_id, yad_no, 1 - dists[i][j]])
word2vec_sim_df = pd.DataFrame(results, columns=['session_id', 'yad_no', 'word2vec_sim'])

In [18]:
word2vec_sim_df

,session_id,yad_no,word2vec_sim
0,000007603d533d30453cc45d0f3d119f,11882,1.000000
1,000007603d533d30453cc45d0f3d119f,4101,1.000000
2,000007603d533d30453cc45d0f3d119f,3324,1.000000
3,000007603d533d30453cc45d0f3d119f,8668,0.997854
4,000007603d533d30453cc45d0f3d119f,5289,0.997854
...,...,...,...
8841301,fffffa7baf370083ebcdd98f26a7e31a,11273,0.465559
8841302,fffffa7baf370083ebcdd98f26a7e31a,846,0.408497
8841303,fffffa7baf370083ebcdd98f26a7e31a,11984,0.397826
8841304,fffffa7baf370083ebcdd98f26a7e31a,10095,0.393421


In [19]:
def make_candidate_word2vec(session_only_df: pd.DataFrame, word2vec_sim_df: pd.DataFrame):
    session_df = session_only_df.merge(word2vec_sim_df, on=['session_id'], how='left')

    session_df = session_df[session_df['yad_no'].notnull()].reset_index(drop=True)
    session_df['yad_no'] = session_df['yad_no'].astype(int)

    session_df['rank'] = session_df.groupby('session_id')['word2vec_sim'].rank(ascending=False, method='min')

    pred_df = session_df.groupby('session_id')['yad_no'].apply(lambda x: x.tolist()).reset_index()
    pred_df = pd.merge(session_only_df, pred_df, on='session_id', how='left')

    pred_df['yad_no'] = pred_df['yad_no'].apply(lambda d: d if isinstance(d, list) else [])

    return session_df, pred_df

In [20]:
train_session_df, train_pred_df = make_candidate_word2vec(train_session_only_df, word2vec_sim_df)
test_session_df, test_pred_df = make_candidate_word2vec(test_session_only_df, word2vec_sim_df)

In [21]:
train_session_df

,session_id,yad_no,word2vec_sim,rank
0,000007603d533d30453cc45d0f3d119f,11882,1.000000,1.0
1,000007603d533d30453cc45d0f3d119f,4101,1.000000,2.0
2,000007603d533d30453cc45d0f3d119f,3324,1.000000,2.0
3,000007603d533d30453cc45d0f3d119f,8668,0.997854,4.0
4,000007603d533d30453cc45d0f3d119f,5289,0.997854,4.0
...,...,...,...,...
5518420,fffffa7baf370083ebcdd98f26a7e31a,11273,0.465559,15.0
5518421,fffffa7baf370083ebcdd98f26a7e31a,846,0.408497,16.0
5518422,fffffa7baf370083ebcdd98f26a7e31a,11984,0.397826,17.0
5518423,fffffa7baf370083ebcdd98f26a7e31a,10095,0.393421,18.0


In [22]:
"""
cos_sims = []
for session_id, yad_no in tqdm(zip(train_session_df['session_id'].values, train_session_df['yad_no'].values)):
    session_last_yad_no = user_last_item_dict[session_id]

    sim = cos_sim(item_factors[session_last_yad_no], item_factors[yad_no])
    cos_sims.append(sim)
"""

"\ncos_sims = []\nfor session_id, yad_no in tqdm(zip(train_session_df['session_id'].values, train_session_df['yad_no'].values)):\n    session_last_yad_no = user_last_item_dict[session_id]\n\n    sim = cos_sim(item_factors[session_last_yad_no], item_factors[yad_no])\n    cos_sims.append(sim)\n"

In [23]:
train_pred_df

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,"[11882, 4101, 3324, 8668, 5289, 2808, 13697, 5..."
1,0000ca043ed437a1472c9d1d154eb49b,"[7609, 3364, 4488, 8253, 3725, 9468, 1586, 287..."
2,0000d4835cf113316fe447e2f80ba1c8,"[602, 7, 363, 580, 622, 283, 558, 673, 180, 33..."
3,0000fcda1ae1b2f431e55a7075d1f500,"[2272, 9790, 11715, 626, 1341, 755, 11803, 943..."
4,000104bdffaaad1a1e0a9ebacf585f33,"[1284, 5490, 7240, 902, 9067, 96, 4072, 7749, ..."
...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,"[13210, 13794, 540, 1678, 554, 10522, 13717, 7..."
288694,ffff2360540745117193ecadcdc06538,"[4767, 3940, 513, 2900, 681, 8465, 5299, 963, ..."
288695,ffff7fb4617164b2604aaf51c40bf82d,"[10364, 4040, 7820, 2097, 7308, 5037, 3566, 12..."
288696,ffffcd5bc19d62cad5a3815c87818d83,"[1820, 12829, 6138, 6933, 12781, 12500, 6595, ..."


## cv
dim=256 0.20479048683487233  
dim=512 0.24105351563621108  
dim=1024 0.284614876566241

In [24]:
mapk(train_df['yad_no'].to_list(), train_pred_df['yad_no'].to_list(), k=10)

0.25015014728963214

In [25]:
train_df

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,4101
1,0000ca043ed437a1472c9d1d154eb49b,8253
2,0000d4835cf113316fe447e2f80ba1c8,4863
3,0000fcda1ae1b2f431e55a7075d1f500,1652
4,000104bdffaaad1a1e0a9ebacf585f33,96
...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,2259
288694,ffff2360540745117193ecadcdc06538,963
288695,ffff7fb4617164b2604aaf51c40bf82d,13719
288696,ffffcd5bc19d62cad5a3815c87818d83,10619


In [26]:
train_session_df.to_pickle(f'../datasets/candidate_df/{OUTPUT_NAME}_train.pkl')
test_session_df.to_pickle(f'../datasets/candidate_df/{OUTPUT_NAME}_test.pkl')

In [27]:
import os
os.makedirs(f'../datasets/{OUTPUT_NAME}_pkl', exist_ok=True)
user_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_user_factors_df.pkl')
user_factors_last_item_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_user_factors_last_item_df.pkl')
item_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_item_factors_df.pkl')